# Part I. ETL Pipeline for Pre-Processing the Files

#### Importing Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from decimal import Decimal
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
print('Printing current working directory')
print(os.getcwd())

print('Getting filepath to the event data files')
filepath = os.getcwd() + '/event_data'

print('Getting the list of all files path')
for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

Printing current working directory
/home/workspace
Getting filepath to the event data files
Getting the list of all files path
['/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-14-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-26-e

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
full_data_rows_list = [] 

for f in file_path_list:

# Reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # Extracting each data row one by one and appending it        
        for line in csvreader:
            full_data_rows_list.append(line) 
print(len(full_data_rows_list))

# Creating a smaller event data csv file called event_datafile_new csv that will be used to insert data
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


1687


In [4]:
# Checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1432


# Part II. Completing the Apache Cassandra coding portion of this project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [5]:
# Making connection to Cassandra instance

from cassandra.cluster import Cluster
cluster = Cluster()

# Getting session
session = cluster.connect()

#### Creating Keyspace

In [6]:

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)


#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


# Function to run query without any parameter
def execute_query(query):
    try:
        return session.execute(query)
    except Exception as e:
        print(e)
        
        
# Function to run query with parameters 
def execute_query_with_params(query, params):
    try:
        return session.execute(query, params)
    except Exception as e:
        print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries we want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Creating Table for Query 1

#### Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

#### Since we want to run query which will filter based on sessionId and ItemInSession we would make sessionId as partition key and ItemInSession as Clustering column

In [8]:

execute_query("""
    CREATE TABLE IF NOT EXISTS session_library (
        session_id int,
        item_in_session int,
        artist text,
        song text,
        song_length decimal,
        PRIMARY KEY (session_id, item_in_session)
    )
""")

                    

#### Checking the number of rows in the CSV file

In [9]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1432


#### Inserting the data in session_library table for Query 1

In [10]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_library (session_id,item_in_session,artist,song,song_length)"
        query = query + "values(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],Decimal(line[5])))

#### Verifying that the data have been inserted into session_library table for Query 1

In [11]:
rows = execute_query("SELECT session_id, item_in_session, artist, song, song_length FROM session_library WHERE session_id = 338 AND item_in_session  = 4")
tb = PrettyTable(['# Table1.Session', '# Table1.Session Item', 'Table1.Artist', 'Table1.Song', 'Table1.Song Duration'])
for row in rows:
    tb.add_row([row.session_id, row.item_in_session, row.artist, row.song, row.song_length])
print(tb)




+------------------+-----------------------+---------------+---------------------------------+----------------------+
| # Table1.Session | # Table1.Session Item | Table1.Artist |           Table1.Song           | Table1.Song Duration |
+------------------+-----------------------+---------------+---------------------------------+----------------------+
|       338        |           4           |   Faithless   | Music Matters (Mark Knight Dub) |       495.3073       |
+------------------+-----------------------+---------------+---------------------------------+----------------------+


## Creating Table for Query 2

#### Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### Since we want to filter query based on userid and session id, we would make userid and session id as partition key because we want different session of the same user to be distributed on the same node. We want the result to be sorted by itemInSession so we would make it as a Clustering column

In [12]:

execute_query("""
    CREATE TABLE IF NOT EXISTS user_library (
        user_id int, 
        session_id int,
        item_in_session int,
        artist text,
        song text,
        first_name text,
        last_name text,
        PRIMARY KEY ((user_id, session_id), item_in_session)
    )
""")
                    

#### Inserting the data in user_library table for Query 2

In [13]:


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_library (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))



                    

#### Verifying that the data have been inserted into user_library table for Query 2

In [14]:

rows = execute_query("SELECT artist, song, first_name, last_name, item_in_session FROM user_library WHERE user_id = 10 AND session_id =182")

tb = PrettyTable(['# Table2.Session Item', '# Table2.Artist', '# Table2.Song', '# Table2.First Name', '# Table2.Last Name'])
for row in rows:
    tb.add_row([row.item_in_session, row.artist, row.song, row.first_name, row.last_name])
print(tb)


                    

+-----------------------+-------------------+------------------------------------------------------+---------------------+--------------------+
| # Table2.Session Item |  # Table2.Artist  |                    # Table2.Song                     | # Table2.First Name | # Table2.Last Name |
+-----------------------+-------------------+------------------------------------------------------+---------------------+--------------------+
|           0           |  Down To The Bone |                  Keep On Keepin' On                  |        Sylvie       |        Cruz        |
|           1           |    Three Drives   |                     Greece 2000                      |        Sylvie       |        Cruz        |
|           2           | Sebastien Tellier |                      Kilometer                       |        Sylvie       |        Cruz        |
|           3           |   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |        Sylvie       |        Cruz  

## Creating Table for Query 3

#### Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### We will need to obtain (select) the user first name and lastname from out table, and we will need to filter by song name. As user name and lastname, in large datasets, are not unique, we will add the column userId to uniquely identify users.

In [15]:
execute_query("""
    CREATE TABLE IF NOT EXISTS user_song_library (
        user_id int, 
        song text,
        artist text,
        first_name text,
        last_name text,
        PRIMARY KEY (song, user_id)
    )
""")
                    

#### Inserting the data in song_library table for Query 3

In [16]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song_library (user_id, song, artist, first_name, last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        queryParameters = (int(line[10]), line[9], line[0], line[1], line[4])
        session.execute(query,queryParameters)

#### Verifying that the data have been inserted into song_library table for Query 3

In [17]:
rows = execute_query("SELECT first_name, last_name, song, artist FROM user_song_library WHERE song = 'All Hands Against His Own'")

t = PrettyTable(['# Table3.First Name', '# Table3.Last Name', '# Table3.Artist', '# Table3.Song'])
for row in rows:
    t.add_row([row.first_name, row.last_name, row.song, row.artist])
print(t)

+---------------------+--------------------+---------------------------+----------------+
| # Table3.First Name | # Table3.Last Name |      # Table3.Artist      | # Table3.Song  |
+---------------------+--------------------+---------------------------+----------------+
|         Sara        |      Johnson       | All Hands Against His Own | The Black Keys |
+---------------------+--------------------+---------------------------+----------------+


### Dropping the tables before closing out the sessions

In [18]:
execute_query("drop table user_library")
execute_query("drop table session_library")
execute_query("drop table user_song_library")

### Closing the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()